# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [ ]:
#import libraries
import os
import cv2
import json
import re
import datetime
import time
import platform
import numpy as np
import pandas as pd
import shutil

In [ ]:
def get_video_duration(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)

    # Get the frames per second (fps) of the video
    fps = video_capture.get(cv2.CAP_PROP_FPS)

    # Calculate the duration in seconds
    duration_seconds = round(total_frames / fps)

    # Convert the duration to a human-readable format (HH:MM:SS)
    video_time = datetime.timedelta(seconds=duration_seconds)

    # Close the video capture object
    video_capture.release()

    return f"duration in seconds: {duration_seconds}, video time: {video_time}"


In [ ]:
def count_frames(video_path):
    video = cv2.VideoCapture(video_path)
    success, frame = video.read()
    frame_count = 0

    while success:
        success, frame = video.read()
        
        if not success:
            break

        timestamp_ms = video.get(cv2.CAP_PROP_POS_MSEC)
        timestamp_sec = timestamp_ms / 1000.0
        print("time: ", timestamp_sec, "frame: ", frame_count)
        print(success, frame)
        frame_count += 1

    print("Manual Frame Count: ", frame_count)
    video.release()

In [ ]:
def re_sample(old_fps: int, new_fps: int) -> list[int]:
    frames_arr = np.arange(0, old_fps, dtype=int)
    frames_interp = np.linspace(0, old_fps - 1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [ ]:
def create_video_from_png(png_folder, output_video_path, fps=30):
    png_files = [f for f in os.listdir(png_folder) if f.endswith('.png')]
    
    if not png_files:
        print(f"No PNG files found in the folder '{png_folder}'.")
        return

    png_files.sort()

    first_image = cv2.imread(os.path.join(png_folder, png_files[0]))
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for AVI format on Windows
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for png_file in png_files:
        img_path = os.path.join(png_folder, png_file)
        img = cv2.imread(img_path)
        out.write(img)

    out.release()
    cv2.destroyAllWindows()
    
    print(f"Video successfully created at '{output_video_path}'.")

In [ ]:
def get_creation_time(file_path):
    if platform.system() == 'Windows':
        # For Windows, use the GetFileTime function
        file_time = os.path.getctime(file_path)
    else:
        # For other platforms like macOS or Linux, use the stat function
        stat = os.stat(file_path)
        file_time = stat.st_ctime

    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(file_time))

In [ ]:
def move_png(log_file):
    with open(log_file, "r") as log_data:
        video_files = log_data.read().split("\n")
    print(video_files[0])

    for video in video_files:
        str_list = video.split("/")
        folder_name = "/".join(str_list[0:-1])
        video_filename = str_list[-1]
        output_folder = folder_name + "/" + "frames"

        if(video_filename.endswith("png")):
            print(folder_name, video_filename)

            if (not os.path.exists(output_folder)):
                os.makedirs(output_folder)

            try:
                shutil.move(video, output_folder)
            except Exception as e:
                print(f"{e}")
            

In [ ]:
log_file = r"C:\Users\chris\Documents\Projects Outside of School\repo\Video-Processing\records\logs\filenames.log"
move_png(log_file)

In [ ]:
# Example usage:
png_folder_path = r"E:\DI_centre_structured\DI_THERMAL_FLIR\15\15_2\15_2nd_2meters\15_2nd_hold_blanket_2meters\15_frames"
output_video_path = r"E:\DI_centre_structured\DI_THERMAL_FLIR\15\15_2\15_2nd_2meters\15_2nd_hold_blanket_2meters\output_video.mp4"
create_video_from_png(png_folder_path, output_video_path)

In [ ]:
file_path = r"E:\DI_centre_structured\DI_THERMAL_FLIR\15\15_2\15_2nd_2meters\15_2nd_hold_blanket_2meters\15_frames\2nd_try-12232022094224-0.png"

try:
    creation_time = get_creation_time(file_path)
    print(f"File creation time: {creation_time}")
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
file_path = r"E:\DI_centre_structured\DI_THERMAL_FLIR\15\15_2\15_2nd_2meters\15_2nd_hold_blanket_2meters\15_frames\2nd_try-12232022094334-2113.png"

try:
    creation_time = get_creation_time(file_path)
    print(f"File creation time: {creation_time}")
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"An error occurred: {e}")